#### Vanessa Navarro Coronado e Iván Sánchez Castellanos

# Práctica 3: Multiclasificadores y selección de variables

En esta práctica vamos a implementar un ensemble genérico, y comparar sus resultados con los obtenidos con los ensembles de la librería de scikit, además del clasificador base DecisionTree.

Además, también utilizaremos distintas técnicas de selección de variables, y las intentaremos aplicar en nuestro ensemble para comprobar si los resultados mejoran.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt
# Cargamos el arbol de decision
from sklearn import tree, base
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
import random

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6342
np.random.seed(seed)

## Carga de datos

#### Wisconsin

Cargamos el dataset Wisconsin de la librería scikit.

In [4]:
from sklearn.datasets import load_breast_cancer
wisconsin = load_breast_cancer()

Separamos en dataset en atributos y clase, pero esta vez utilizaremos las versiones numéricas de los datos.

In [5]:
attributesWisc = wisconsin.data
labelWisc = wisconsin.target

Con este dataset, realizamos la separación de los datos en Train y Test.

In [6]:
train_attsWisc, test_attsWisc, train_labelWisc, test_labelWisc = train_test_split( 
    attributesWisc,
    labelWisc,
    test_size=0.2,
    random_state=seed,
    stratify=labelWisc)

#### Pima

Para el dataset Pima, no podemos usar el que proporciona scikit porque es para un problema de regresión, por lo que lo incluiremos usando el csv de Campus Virtual.

In [7]:
dfPima = pd.read_csv("../data/pima.csv", dtype={ "label": 'category'})
attributesPima = dfPima.drop('label', 1)
labelPima = dfPima['label']

Convertimos Pima en numpy array.

In [8]:
attsPima=attributesPima.values
labPima=np.array(labelPima.cat.codes)

Con este dataset, realizamos la separación de los datos en Train y Test.

In [9]:
train_attsPima, test_attsPima, train_labelPima, test_labelPima = train_test_split( 
    attsPima,
    labPima,
    test_size=0.2,
    random_state=seed,
    stratify=labelPima)

# 1. Implementación básica de un ensemble de manera manual

Para implementar un ensemble de forma similar a los ensembles de la librería Scikit, debemos crear una clase que herede de base.BaseEstimator, la cual ya presenta algunas funciones básicas como get_params y set_params, que son necesarias para su correcto funcionamiento.

Una vez hecho esto, debemos inicializar los atributos de nuestro ensemble. Estos son:
* estim: Estimador para el ensemble.
* nEstim: Número de estimadores que usaremos en nuestro ensemble.
* replace: Atributo booleano para indicar si el muestreo de los casos se realzará con reemplazo (True) o sin reemplazo (False). Por defecto, este atributo está inicializado a False.
* attSample: Atributo booleano para indicar si se va a realizar muestreo de atributos (True) o no (False). Por defecto, este atributo está inicializado a False.
* dataFrac: Fracción de los casos que tendremos en cuenta a la hora de realizar el muestreo. Por defecto, estará a un 60%.
* attFrac: Fracción de los atributos que tendremos en cuenta a la hora de realizar el muestreo. Por defecto, estará a un 80%.
* randomState: Semilla que usaremos para los muestreos.
* models: Vector de modelos del ensemble.
* atts: Vector de atributos a tener en cuenta.
* predictions: Vector de predicciones realizadas.

La siguiente función a implementar es la función fit, que se encarga del entrenamiento de los estimadores de nuestro ensemble. Como resultado de dicho entrenamiento, cada modelo entrenado será almacenado en un vector de modelos. Esta función también se encarga de realizar los correspondientes muestreos tanto de casos como de atributos, previos al entrenamiento en sí de los modelos.

La función predict se encarga de realizar una predicción con cada uno de los modelos aprendidos en la fase anterior. Cada una de las predicciones será almacenada en el vector de predicciones del ensemble, y posteriormente deberá realizar un voto por mayoría para devolver la clase mayoritaria en cada uno de los casos del dataset.


Como vemos, nuestro ensemble es genérico, y similar al Bagging, ya que en cada iteración, realiza un muestreo de los casos, entrena un modelo con él, y por último genera una predicción.

In [10]:
class OurEnsemble(base.BaseEstimator):
    
    def __init__(self, estim, nEstim, randomState, dataFrac=0.6, replace=False, attSample=False, attFrac=0.8):
        self.estim=estim #estimador
        self.nEstim=nEstim #numero de estimadores
        self.replace=replace #para indicar si el muestreo sera con reemplazo o sin reemplazo (sin reemplazo por defecto)
        self.attSample=attSample #para indicar si haremos muestreo de atributos (por defecto esta desactivado)
        self.dataFrac = dataFrac #porcentaje de casos para el muestreo (60% por defecto)
        self.randomState=randomState #semilla para el muestreo
        self.attFrac=attFrac #porcentaje de atributos que tendremos en cuenta (80% por defecto)
        
    def fit(self,trainAtts,trainLab): #funcion para entrenar los modelos
        self.models=[] #vector de modelos
        self.atts=[] #vector de atributos
        
        np.random.seed(self.randomState) #establecemos la semilla
        
        #convertir el dataset a un dataframe de pandas para que sea mas facil de manejar
        trainAtts=pd.DataFrame(trainAtts) 
        trainLab=pd.DataFrame(trainLab)
        
        for n in range(0,self.nEstim):  
            #hacer muestreo de los casos
            trainAttsSample=trainAtts.sample(frac=self.dataFrac,replace=self.replace)
            
            #si esta activo el muestreo de atributos...
            if self.attSample:
                #almacenamos en una lista las columnas o atributos del dataset
                #multiplicamos el procentaje de atributos a tener en cuenta por el numero total de atributos
                #obtenemos una muestra de los atributos con la funcion random.choice
                attMuestreo=np.random.choice(list(trainAtts.columns.values), 
                                             size=int(self.attFrac*len(trainAtts.columns)), 
                                             replace=False)
                #añadimos los atributos resultantes del muestreo a nuestro vector de atributos
                self.atts.append(attMuestreo)
                
                #nos quedamos con los casos del dataset cuyas columnas o atributos han sido elegidas en el muestreo de atributos
                trainAttsSample=trainAttsSample[attMuestreo]
                
            #continuamos con el muestreo de los casos:
            #ahora debemos coger los indices de los casos seleccionados en el muestreo, 
            #para poder coger esos mismos casos en las labels del dataset
            trainLabelSample=trainLab.loc[trainAttsSample.index.values]
            
            #clonamos el estimador para que el objeto no sufra cambios al ir añadiendo el modelo aprendido al vector de modelos
            e=base.clone(self.estim)
            
            #entrenamos el estimador con los muestreos de casos y de atributos realizados
            e.fit(trainAttsSample,trainLabelSample)
            
            #añadimos el modelo entrenado al vector de modelos
            self.models.append(e)
            
        
    def predict(self,testAtts):
        self.predictions=[] #vector de predicciones de cada modelo
        
        for n in range(0,self.nEstim):
            #guardamos los atributos del test en una variable para no modificar el original
            testAttsSample = testAtts
            
            #si el muestreo de atributos esta activado...
            if self.attSample:
                #seleccionamos todos los casos del test, pero solo cogemos los atributos obtenidos tras el muestreo correspondiente
                testAttsSample=testAttsSample[:,self.atts[n]]
            
            #realizamos la prediccion con el modelo entrenado
            pred=self.models[n].predict(testAttsSample)
            #añadimos la prediccion a nuestro vector de predicciones
            self.predictions.append(pred)
        
        #Para realizar la prediccion general, realizaremos un voto por mayoria de todas las prediciones, por eso usamos la moda
        moda=stats.mode(self.predictions)
        #devolvemos el resultado de aplicar la moda a las predicciones
        return moda.mode[0]
        
        

Una vez creado nuestro ensemble genérico, lo probamos.

In [11]:
ens = OurEnsemble(tree.DecisionTreeClassifier(random_state=seed), nEstim = 10, randomState=seed, attSample=True)

In [12]:
scores_ensWisc = cross_val_score(ens, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy Our Ensemble Wisc: %0.2f (+/- %0.2f)" % (scores_ensWisc.mean(), scores_ensWisc.std() * 2))

Accuracy Our Ensemble Wisc: 0.94 (+/- 0.05)


In [13]:
scores_ensPima = cross_val_score(ens, attsPima, labPima, cv=3, scoring="accuracy")
print("Accuracy Our Ensemble Pima: %0.2f (+/- %0.2f)" % (scores_ensPima.mean(), scores_ensPima.std() * 2))

Accuracy Our Ensemble Pima: 0.74 (+/- 0.05)


## 1.1. Árbol de decisión como referencia

In [14]:
dt = tree.DecisionTreeClassifier(random_state=seed)
scores_dtW = cross_val_score(estimator=dt, X=attributesWisc, y=labelWisc, scoring="accuracy", cv=3)
print("Accuracy Tree Wisc: %0.2f (+/- %0.2f)" % (scores_dtW.mean(), scores_dtW.std() * 2))

Accuracy Tree Wisc: 0.90 (+/- 0.07)


In [15]:
scores_dtP = cross_val_score(estimator=dt, X=attsPima, y=labPima, scoring="accuracy", cv=3)
print("Accuracy Tree Pima: %0.2f (+/- %0.2f)" % (scores_dtP.mean(), scores_dtP.std() * 2))

Accuracy Tree Pima: 0.71 (+/- 0.06)


## 1.2. Bagging

W: Wisconsin

P: Pima

In [16]:
from sklearn.ensemble import BaggingClassifier

bagg = BaggingClassifier(tree.DecisionTreeClassifier(), n_estimators = 30, random_state=seed)

In [17]:
scores_baggW = cross_val_score(bagg, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy Bagging W: %0.2f (+/- %0.2f)" % (scores_baggW.mean(), scores_baggW.std() * 2))

Accuracy Bagging W: 0.93 (+/- 0.04)


In [18]:
scores_baggP = cross_val_score(bagg, attsPima, labPima, cv=3, scoring="accuracy")
print("Accuracy Bagging P: %0.2f (+/- %0.2f)" % (scores_baggP.mean(), scores_baggP.std() * 2))

Accuracy Bagging P: 0.75 (+/- 0.02)


## 1.3. Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, random_state=seed)

In [20]:
scores_rfW = cross_val_score(rf, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy Random Forest W: %0.2f (+/- %0.2f)" % (scores_rfW.mean(), scores_rfW.std() * 2))

Accuracy Random Forest W: 0.95 (+/- 0.03)


In [21]:
scores_rfP = cross_val_score(rf, attsPima, labPima, cv=3, scoring="accuracy")
print("Accuracy Random Forest P: %0.2f (+/- %0.2f)" % (scores_rfP.mean(), scores_rfP.std() * 2))

Accuracy Random Forest P: 0.76 (+/- 0.03)


## 1.4. Boosting

In [22]:
from sklearn.ensemble import AdaBoostClassifier
#boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=30, random_state=seed) #accuracy: 0.90 (+/- 0.08)
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=3), n_estimators=30, random_state=seed)

In [23]:
scores_boostW = cross_val_score(boost, attributesWisc, labelWisc, cv=3, scoring="accuracy")
print("Accuracy Boosting W: %0.2f (+/- %0.2f)" % (scores_boostW.mean(), scores_boostW.std() * 2))

Accuracy Boosting W: 0.97 (+/- 0.03)


In [24]:
scores_boostP = cross_val_score(boost, attsPima, labPima, cv=3, scoring="accuracy")
print("Accuracy Boosting P: %0.2f (+/- %0.2f)" % (scores_boostP.mean(), scores_boostP.std() * 2))

Accuracy Boosting P: 0.71 (+/- 0.03)


## 1.5. Gradient Boosting

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
gboost = GradientBoostingClassifier(n_estimators=30, random_state=seed)

In [26]:
scores_gboostW = cross_val_score(gboost, attributesWisc, labelWisc, cv=5, scoring="accuracy")
print("Accuracy Gradient Boosting W: %0.2f (+/- %0.2f)" % (scores_gboostW.mean(), scores_gboostW.std() * 2))

Accuracy Gradient Boosting W: 0.95 (+/- 0.03)


In [27]:
scores_gboostP = cross_val_score(gboost, attsPima, labPima, cv=5, scoring="accuracy")
print("Accuracy Gradient Boosting W: %0.2f (+/- %0.2f)" % (scores_gboostP.mean(), scores_gboostP.std() * 2))

Accuracy Gradient Boosting W: 0.76 (+/- 0.07)


## Comparación
Ahora vamos a mostrar una comparativa de  los resultados obtenidos con nuestro ensemble, y los demás ensembles de scikit.

In [28]:
# How to make a HTML table!
from IPython.display import display, HTML

def printTable(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % (cell % x[0] + cell % x[1]) for x in results])
    display(HTML(report))
    

In [29]:
results = (("Decision Tree", scores_dtW.mean()), 
           ("Bagging", scores_baggW.mean()), 
           ("Random Forest", scores_rfW.mean()),
           ("Boosting", scores_boostW.mean()),
           ("Gradient Boosting", scores_gboostW.mean()),
            ("Our Ensemble", scores_ensWisc.mean()))

print('Wisconsin')
printTable(results)

Wisconsin


Decision Tree,0.897994987469
Bagging,0.934939199851
Random Forest,0.954311705189
Boosting,0.971864847303
Gradient Boosting,0.95444401693
Our Ensemble,0.942003156038


Como vemos, el algoritmo Boosting sigue siendo el mejor, pero nuestro ensemble genérico supera al clasificador base, y también al Bagging.

In [30]:
results = (("Decision Tree", scores_dtP.mean()), 
           ("Bagging", scores_baggP.mean()), 
           ("Random Forest", scores_rfP.mean()),
           ("Boosting", scores_boostP.mean()),
           ("Gradient Boosting", scores_gboostP.mean()),
            ("Our Ensemble", scores_ensPima.mean()))

print('Pima')
printTable(results)

Pima


Decision Tree,0.708248852394
Bagging,0.749982138202
Random Forest,0.76038364519
Boosting,0.713594695741
Gradient Boosting,0.757711569476
Our Ensemble,0.740885416667


Como vemos, el algoritmo Random Forest sigue siendo el mejor, pero nuestro ensemble genérico supera al clasificador base, y también al Boosting.

# 2. Selección de variables

## 2.1. Utilizar un método filter basado en rankings y la importancia de las variables para evaluar distintos subconjuntos

Para realizar esta selección de variables, vamos usar una función de la librería de scikit llamada SelectKBest, que se encarga de seleccionar, de acuerdo a diferentes criterios, los K mejores atributos de un dataset determinado. Entre los criterios de selección que utiliza este algoritmo, están f_classif, que realiza un test F o ANOVA estadístico, chi2, que realiza el test de chi cuadrado, o mutual_info_classif, que selecciona las variables de acuerdo a su información mutua respecto a la clase. 

Para probarlo, vamos a llamar a la función con el criterio de Información Mutua, y vamos a decirle que nos devuelva un ranking de las 10 mejores variables. Una vez seleccionados los atributos, debemos llamar a la función fit_transform, para entrenar con los datos, y después seleccionar solo los K atributos elegidos con mayor score.

Para Pima, como tiene menos variables, usaremos otras cifras más bajas.

In [31]:
from sklearn.feature_selection import SelectKBest,f_classif, mutual_info_classif, chi2

In [32]:
sW=SelectKBest(score_func=mutual_info_classif, k=10)
bestAttributesWisc = sW.fit_transform(attributesWisc, labelWisc)

In [33]:
sP=SelectKBest(score_func=mutual_info_classif, k=5)
bestAttributesPima = sP.fit_transform(attsPima, labPima)

Ahora vamos a usar un GridSearch para comprobar cuál es el criterio de selección que mejor score presentará. Para ello, vamos a crear un Pipeline, donde el primer paso será la selección de variables, y el segundo paso será nuestro ensemble. 

In [34]:
from sklearn.pipeline import Pipeline

In [35]:
estimatorENS = Pipeline([("Ranker", SelectKBest()),
                      ("Ensemble", OurEnsemble(
                          tree.DecisionTreeClassifier(random_state=seed), 
                          nEstim = 10, 
                          randomState=seed ) )])

Después creamos un objeto de la clase GridSearchCV, donde el estimador será nuestro pipeline, y su param_grid estará formado por los criterios de selección de variables f_classif, chi2 y mutual_info_classif, explicados anteriormente.

In [36]:
from sklearn.model_selection import GridSearchCV

skbW = GridSearchCV(
    estimator = estimatorENS,
    param_grid = 
        { 'Ranker__score_func' : [f_classif, chi2, mutual_info_classif], 'Ranker__k': [5,10] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False
)

In [37]:
skbP = GridSearchCV(
    estimator = estimatorENS,
    param_grid = 
        { 'Ranker__score_func' : [f_classif, chi2, mutual_info_classif], 'Ranker__k': [3,5] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False
)

Entrenamos y realizamos las predicciones, obteniendo el accuracy. Como vemos, ha mejorado considerablemente con respecto al comienzo de la práctica. Sin embargo, en Pima el accuracy desciende.

In [38]:
import sklearn.metrics as metrics

fittedW=skbW.fit(train_attsWisc, train_labelWisc)
predW=skbW.predict(test_attsWisc)
print('Accuracy OurEnsemble + Ranker W:')
metrics.accuracy_score(test_labelWisc, predW)

Accuracy OurEnsemble + Ranker W:


0.97368421052631582

In [39]:
fittedP=skbP.fit(train_attsPima, train_labelPima)
predP=skbP.predict(test_attsPima)
print('Accuracy OurEnsemble + Ranker P:')
metrics.accuracy_score(test_labelPima, predP)

Accuracy OurEnsemble + Ranker P:


0.72727272727272729

A continuación mostramos la mejor configuración de parámetros obtenida para el Ranker usando el GridSearch creado previamente. Esta es usar un número de variables k=10, y usar la función del test F como criterio de selección de variables. Y para Pima, es usar 5 variables y la función de Información Mutua.

In [40]:
print('Mejor configuración de parámetros Ranker W:')
fittedW.best_params_

Mejor configuración de parámetros Ranker W:


{'Ranker__k': 10,
 'Ranker__score_func': <function sklearn.feature_selection.univariate_selection.f_classif>}

In [41]:
print('Mejor configuración de parámetros Ranker P:')
fittedP.best_params_

Mejor configuración de parámetros Ranker P:


{'Ranker__k': 5,
 'Ranker__score_func': <function sklearn.feature_selection.mutual_info_.mutual_info_classif>}

## 2.2. Implementar al menos un algoritmo de búsqueda recursiva wrapper

Para implementar el Wrapper, debemos tener en cuenta que los subconjuntos de atributos que vamos a utilizar serán aleatorios, por lo que necesitaremos un atributo que nos diga cuándo parar de hacer iteraciones. Para ello, vamos a crear una clase que tendrá como atributos:
* estim: Estimador que usaremos para aprender modelos y evaluar así los subconjuntos de variables.
* nIter: Número de iteraciones que vamos a realizar. En cada iteración evaluaremos un subconjunto de variables distinto.
* randomState: Semilla para la selección de atributos.
* nAtts: Número de atributos que tendrán nuestros subconjuntos.
* bestAtts: lista con los mejores atributos seleccionados.
* maxScore: mejor score obtenido en la validación cruzada.

Una vez inicializados los atributos, vamos a definir una función findBestAtts que se encargará de hacer la selección de variables. Primero realizamos un bucle for para realizar el muestreo de los atributos. El segundo bucle for es para obtener los scores de la validación cruzada realizada con nuestro estimador y el conjunto de atributos seleccionado. De esta validación cruzada, nos quedaremos con el subconjunto de variables que mejor score obtenga.

In [42]:
class OurWrapper(object):
    def __init__(self, estim, nIter, randomState, nAtts):
        self.estim=estim #estimador
        self.nIter=nIter #numero de iteraciones a realizar o numero de subconjuntos de variables a analizar
        self.randomState=randomState #semilla
        self.nAtts=nAtts #numero de variables en el subconjunto
        
    def findBestAtts(self, trainAtts, trainLab):
        self.bestAtts=[] #vector de mejores atributos encontrados
        self.maxScore=0 #para ir guardando el mejor score obtenido
        attMuestreo=[] #vector auxiliar para ir guardando los subconjuntos de variables obtenidos
        
        np.random.seed(self.randomState) #establecemos la semilla
        
        #bucle para realizar los subconjuntos aleatorios de atributos
        for i in range(0,self.nIter):
            attMuestreo.append(np.random.choice(range(0,trainAtts.shape[1]), 
                                                size=self.nAtts, 
                                                replace=False))
            #print(attMuestreo[i])
        
        #bucle para realizar la validacion cruzada con nuestro estimador y los atributos seleccionados
        for i in range(0,self.nIter):
            #el conjunto de train estara formado por todos los casos, pero solo analizando las variables elegidas
            trainAttsSample=trainAtts[:,attMuestreo[i]]
            
            #realizamos una 5 validacion cruzada con el estimador y los datos con las variables seleccionadas
            score=cross_val_score(self.estim, trainAttsSample, trainLab, cv=5, scoring="accuracy")
            
            #nos quedamos con el subconjunto de atributos que mayor score haya obtenido
            if score.mean()>self.maxScore:
                self.maxScore=score.mean()
                self.bestAtts=attMuestreo[i]
            
            
        

Ahora lo probamos usando nuestro ensemble y 10 subconjuntos de 10 atributos cada uno. Para Pima usaremos cifras más bajas.

In [43]:
wW=OurWrapper(OurEnsemble(tree.DecisionTreeClassifier(random_state=seed), 
                          nEstim = 10, 
                          randomState=seed ),
            10,seed,10)
wW.findBestAtts(train_attsWisc,train_labelWisc)

In [44]:
wP=OurWrapper(OurEnsemble(tree.DecisionTreeClassifier(random_state=seed), 
                          nEstim = 10, 
                          randomState=seed ),
            10,seed,5)
wP.findBestAtts(train_attsPima,train_labelPima)

Mostramos el mejor subconjunto de atributos de nuestro dataset. En este caso los mejores atributos han sido aquellos que se encuentran en las columnas 21,  4,  5,  7, 20, 18, 24, 15, 17 y 19. Y para Pima, las mejores son 7, 1, 0, 5, 3.


In [45]:
wW.bestAtts

array([21,  4,  5,  7, 20, 18, 24, 15, 17, 19])

In [46]:
wP.bestAtts

array([7, 1, 0, 5, 3])

Por último, mostramos el mejor score obtenido con la mejor combinación de atributos. En este caso, mejora muy poco con respecto al comienzo de la práctica, por lo que podemos concluir que lo mejor sería usar el ranker del apartado anterior. Sin embargo, en Pima mejora.

In [47]:
wW.maxScore

0.94725274725274722

In [48]:
wP.maxScore

0.75077968812475004